In [ ]:
import tempfile
from pathlib import Path

import pandas as pd
import py7zr

In [ ]:
# Read the Parquet file

def read_parquet_gzip_7z(filepath: str | Path) -> pd.DataFrame:
    """Read a .parquet.gzip.7z file into a DataFrame.

    Extracts the 7z archive to a temp directory, then reads the inner
    parquet file with pandas. The '.gzip' in the filename refers to
    the parquet-internal compression codec, not external gzip wrapping.
    """
    with tempfile.TemporaryDirectory() as tmpdir:
        with py7zr.SevenZipFile(filepath, mode="r") as archive:
            archive.extractall(path=tmpdir)
        extracted = list(Path(tmpdir).iterdir())
        if not extracted:
            raise ValueError(f"No files found inside {filepath}")
        return pd.read_parquet(extracted[0])


In [ ]:
path = ('../data/5FT0491_2026-01-27.parquet.gzip.7z')
df = read_parquet_gzip_7z(path)

In [ ]:
df.columns.tolist()

# Columns Group By Functions, According to Gemini

## Human Inputs

- `EEC2_Engine.AccelPedalPos1`: The gas pedal position (0-100%). Key for Accel Overrides.

- `EBC1_Brakes.BrakePedalPos`: The brake pedal position. Key for Brake Overrides.

- `CCVS1_Cab_Controller.CruiseCtrl...`: Buttons the driver pressed (Resume, Set, Cancel).

- `Tablet.UserButtonPress`: Likely a custom event tagger (e.g., driver flagging a bug).

## ADAS Planning AKA Behaviors from System

- `ACC1_Radar.AdaptiveCruiseCtrlSetSpeed`: The speed the system was trying to hold.

- `ACC1_Radar.DistanceToForwardVehicle`: The gap it was trying to maintain.

- `ACC1_Radar.ForwardCollisionWarning`: Did the system think a crash was imminent?

- `iQEH3.iQTargSpeed`: The internal target speed calculated by the "intelligent" controller.

- `PCC1.PCCState / ACC1_Radar.AdaptiveCruiseCtrlMode`: The current health/state of the ADAS (Active, Standby, Faulted).

## Perception & Radar

- `ACC1_Radar.TargetDetected`: Boolean (True/False) if it sees a car.

- `ACC1_Radar.SpeedOfForwardVehicle`: Relative speed of the lead car.

- `Radar.Rx_Out_X, Radar.Ry_Out_X`: Raw Radar Targets. Rx is longitudinal distance, Ry is lateral position.

Note: You have _0 through _7. These are individual objects. If Rx_Out_0 suddenly jumps to 5 meters (ghost object) and the car brakes, you found your root cause.

- `IQCU_FCR.RadarMisalign...`: Diagnostic values for whether the radar is tilted (causes "Yo-yoing").

## Localization & Hills

- `IQVS1.iQRoadGrade / VDS_FC.Altitude`: The slope of the road. Key for "Slow approach to hills."

- `IQCU_IMU_Status_4.PathCurvature`: How sharp the curve is. Key for "Too fast in curves."

- `EH_not_localized`: Indicates if the map data (Electronic Horizon) is lost.

- `IQEH1.iQLCASpeedLimCur`: The speed limit the map thinks is active.

## Vehicle Dynamics (Physics)

- `EEC1_Engine.ActualEngPercentTorque`: How hard the engine was working.

- `EBC2_Brakes.FrontAxleSpeed`: Vehicle speed.

- `ACCS_RP.LongitudinalAccelerationExRange`: Actual G-force (did the car jerk?).